<a href="https://colab.research.google.com/github/ElynZeng/QM2-PROJECT/blob/main/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

data_dir = "/content/drive/MyDrive/Data/"

files = {
    "Vietnam": "Vietnam Data.csv",
    "China": "China Data.csv",
    "Indonesia": "Indonesia Data.csv",
    "Korea": "Korea Data.csv",
    "Japan": "Japan Data.csv",
    "Malaysia": "Malaysia Data.csv",
    "Thailand": "Thailand Data.csv"
}

def fill_linear(series, years):
    s = series.copy()
    known = s.dropna()
    if len(known) < 2:
        return s
    y = years.loc[known.index]
    slope = (known.iloc[-1] - known.iloc[0]) / (y.iloc[-1] - y.iloc[0])
    intercept = known.iloc[0] - slope * y.iloc[0]
    s.loc[s.isna()] = slope * years.loc[s.isna()] + intercept
    return s

def fill_population_density(series, years):
    s = series.copy()
    known = s.dropna()
    if len(known) == 0:
        return s
    if len(known) == 1:
        s[:] = known.iloc[0]
        return s
    y = years.loc[known.index]
    slope = (known.iloc[-1] - known.iloc[0]) / (y.iloc[-1] - y.iloc[0])
    intercept = known.iloc[0] - slope * y.iloc[0]
    s.loc[s.isna()] = slope * years.loc[s.isna()] + intercept
    return s

for country, filename in files.items():
    df = pd.read_csv(data_dir + filename, encoding="utf-8")
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
    df = df.sort_values(["AREA", "YEAR"])

    for col in ["GDP PER CAPITA", "POPULATION DENSITY", "TEMPERATURE", "YEAR"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df["GDP PER CAPITA"] = (
        df.groupby("AREA", group_keys=False)
          .apply(lambda g: fill_linear(g["GDP PER CAPITA"], g["YEAR"]))
    )

    df["POPULATION DENSITY"] = (
        df.groupby("AREA", group_keys=False)
          .apply(lambda g: fill_population_density(
              g["POPULATION DENSITY"], g["YEAR"]
          ))
    )

    df["TEMPERATURE"] = (
        df.groupby("AREA", group_keys=False)
          .apply(lambda g: fill_linear(g["TEMPERATURE"], g["YEAR"]))
    )

    for col in ["GDP PER CAPITA", "POPULATION DENSITY", "TEMPERATURE"]:
        df[col] = df[col].round(2)

    df.to_csv(
        f"{data_dir}{country}_Data_cleaned.csv",
        index=False,
        encoding="utf-8-sig"
    )



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1231443352.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_linear(g["GDP PER CAPITA"], g["YEAR"]))
/tmp/ipython-input-1231443352.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_population_density(
/tmp/ipython-input-1231443352.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecate